# Training

In [9]:
import torch.nn as nn
import torch
from tqdm import tqdm
from vae_based_medical_image_generator.data import dataset
from torch.utils.data import DataLoader
from vae_based_medical_image_generator.model.vae import EncoderVAE, DecoderVAE, VariationalAutoencoder, vae_loss

# Define hyperparameters for training
img_channels = 1
feature_dim = 32*20*20
latent_dim = 100

batch_size = 32
lr = 1e-4
num_epochs = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
# initialize the encoder and the decoder

encoder = EncoderVAE(img_channels, feature_dim, latent_dim) 
decoder = DecoderVAE(img_channels, feature_dim, latent_dim)
vae = VariationalAutoencoder(encoder, decoder, device).to(device)

# initialize optimizer
optimizer = torch.optim.Adam(vae.parameters(), lr=lr)

In [11]:
# initialize dataset
train_dataset = dataset.load_dataset(dataset_name="organamnist", split="train")
test_dataset  = dataset.load_dataset(dataset_name="organamnist", split="test")

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=test_dataset,  batch_size=batch_size, shuffle=False)

Using downloaded and verified file: /home/leon/.medmnist/organamnist.npz


Using downloaded and verified file: /home/leon/.medmnist/organamnist.npz


In [17]:
"""
Training the network for a given number of epochs
The loss after every epoch is printed
"""
for epoch in range(num_epochs):
    with tqdm(train_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch}")

        for data in tepoch:
            imgs, _ = data
            imgs = imgs.to(device)
            
            # Feeding a batch of images into the network to obtain the output image, mu, and logVar
            out, mu, logVar = vae(imgs)
            
            # The loss is the BCE loss combined with the KL divergence to ensure the distribution is learnt
            loss = vae_loss(imgs, out, mu, logVar)
            
            # Backpropagation based on the loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tepoch.set_postfix(loss=loss.item())
        

Epoch 9: 100%|██████████| 1081/1081 [00:11<00:00, 96.08batch/s, loss=-4.53e+5]


In [18]:
import matplotlib.pyplot as plt
import numpy as np
import random

vae.eval()
with torch.no_grad():
    for data in random.sample(list(test_loader), 1):
        imgs, _ = data
        imgs = imgs.to(device)
        img = np.transpose(imgs[0].cpu().numpy(), [1,2,0])
        plt.subplot(121)
        plt.imshow(np.squeeze(img))
        out, mu, logVAR = vae(imgs)
        outimg = np.transpose(out[0].cpu().numpy(), [1,2,0])
        plt.subplot(122)
        plt.imshow(np.squeeze(outimg))